# MssqlDB

Examples with dbatools

## Connect to a SQL Instance

In [2]:
# -- Connect-DbaInstance 'localhost:8433' as sa
if([String]::IsNullOrEmpty($sqlcred)){
    $sqlcred = Get-Credential sa
}
$mssqlsrv = Connect-DbaInstance -SqlInstance 'localhost:8433' -SqlCredential $sqlcred

## List all databases

In [13]:
# -- Get-DbaDatabase
Get-DbaDatabase -SqlInstance $mssqlsrv | FT

ComputerName InstanceName SqlInstance Name   Status IsAccessible RecoveryModel LogReuseWaitStatus SizeMB Compatibility
------------ ------------ ----------- ----   ------ ------------ ------------- ------------------ ------ -------------
localhost    MSSQLSERVER  mssqlsrv1   master Normal         True        Simple            Nothing   6.25    Version150
localhost    MSSQLSERVER  mssqlsrv1   tempdb Normal         True        Simple            Nothing     40    Version150
localhost    MSSQLSERVER  mssqlsrv1   model  Normal         True          Full            Nothing     16    Version150


localhost    MSSQLSERVER  mssqlsrv1   msdb   Normal         True        Simple            Nothing ...625    Version150
localhost    MSSQLSERVER  mssqlsrv1   tinu   Normal         True          Full            Nothing     16    Version150




In [8]:
# -- Get-DbaDatabase -ExcludeSystem
Get-DbaDatabase -SqlInstance $mssqlsrv -ExcludeSystem



ComputerName       : localhost
InstanceName       : MSSQLSERVER
SqlInstance        : mssqlsrv1
Name               : tinu
Status             : Normal
IsAccessible       : True
RecoveryModel      : Full
LogReuseWaitStatus : Nothing
SizeMB             : 16
Compatibility      : Version150
Collation          : SQL_Latin1_General_CP1_CI_AS
Owner              : sa
LastFullBackup     : 23.05.2021 10:40:42
LastDiffBackup     : 01.01.0001 00:00:00
LastLogBackup      : 23.05.2021 10:48:29





In [5]:
Find-DbaDatabase -SqlInstance $mssqlsrv -Pattern 'tinu'



ComputerName       : localhost
InstanceName       : MSSQLSERVER
SqlInstance        : localhost,8433
Name               : tinu
Size               : 16.00 MB
Owner              : sa
CreateDate         : 23.05.2021 10:48:46
ServiceBrokerGuid  : 1c6d1231-d404-4f25-9dea-fa2a7a61262b
Tables             : 1
StoredProcedures   : 0
Views              : 1
ExtendedProperties : 0





## Max Memory

MaxValue 2147483647 MB use the dynamically memory management. From the total OS memory, reserve 1GB-4GB to the OS itself. For more information see on [Server memory configuration options](https://docs.microsoft.com/en-us/sql/database-engine/configure-windows/server-memory-server-configuration-options?view=sql-server-ver15)

In [9]:
Get-DbaMaxMemory -SqlInstance $mssqlsrv



ComputerName : localhost
InstanceName : MSSQLSERVER
SqlInstance  : mssqlsrv1
Total        : 5028
MaxValue     : 2147483647





In [10]:
Test-DbaMaxMemory -SqlInstance $mssqlsrv

 1. | No services found in relevant namespaces on sTinu. Please note that this function is available from SQL 2005 up.


ComputerName     : localhost
InstanceName     : MSSQLSERVER
SqlInstance      : mssqlsrv1
InstanceCount    : 1
Total            : 5028
MaxValue         : 2147483647
RecommendedValue : 1956





In [12]:
Set-DbaMaxMemory -SqlInstance $mssqlsrv

 1. | No services found in relevant namespaces on sTinu. Please note that this function is available from SQL 2005 up.




ComputerName     : localhost
InstanceName     : MSSQLSERVER
SqlInstance      : mssqlsrv1
Total            : 5028
MaxValue         : 1956
PreviousMaxValue : 1956





## Export JSON

In [32]:
$DBname     = 'tinu'
$Table      = 'inventory'
$downloads  = Join-Path $env:USERPROFILE -ChildPath downloads
$Backupfile = Join-Path $downloads -ChildPath "$($DBname)-$($Table)-$(Get-Date -f 'yyyy-MM-dd_HH-mm-ss').json"

$SqlQuery   = @"
SELECT * FROM [$DBname].[dbo].[$Table]
GO
"@  

$ResultSqlQuery = Invoke-DbaQuery -SqlInstance $mssqlsrv -Database $DBname -Query $SqlQuery
$ResultSqlQuery | ConvertTo-Json -Depth 1 | Set-Content $Backupfile

In [36]:
$DBname     = 'tinu'
$Table      = 'inventory'
$downloads  = Join-Path $env:USERPROFILE -ChildPath downloads
$Backupfile = Join-Path $downloads -ChildPath "$($DBname)-$($Table)-$(Get-Date -f 'yyyy-MM-dd_HH-mm-ss').csv"

$SqlQuery   = @"
SELECT * FROM [$DBname].[dbo].[$Table]
GO
"@  

$ResultSqlQuery = Invoke-DbaQuery -SqlInstance $mssqlsrv -Database $DBname -Query $SqlQuery
$ResultSqlQuery | ConvertTo-Csv -Delimiter ';' -NoTypeInformation | Set-Content $Backupfile